In [ ]:
# This code is a re-produce of min-char-rnn.py (https://gist.github.com/karpathy/d4dee566867f8291f086)
# Most parts of code are the same, with cleaner format and code style. (Python3)
# For learning purposes only.

In [5]:
# Import libraries
import numpy as np

In [6]:
# Data I/O
data = open('text.txt', 'r').read()
chars = list(set(data))
data_size = len(data)
vocab_size = len(chars)
print('data has {} characters, {} unique.'.format(data_size, vocab_size))
char_to_ix = { ch: i for i, ch in enumerate(chars) }
ix_to_char = { i: ch for i, ch in enumerate(chars) }

data has 5027 characters, 68 unique.


In [7]:
# Hyper-parameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [8]:
# Model parameters
W_xh = np.random.randn(hidden_size, vocab_size) * 0.01 # input to hidden
W_hh = np.random.randn(hidden_size, hidden_size) * 0.01 # hidden to hidden
W_hy = np.random.randn(vocab_size, hidden_size) * 0.01 # hidden to output
b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((vocab_size, 1)) # output bias

In [9]:
def lossFun(inputs, targets, hprev):
    
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in np.arange(len(inputs)):
        # encode input in 1-of-k representation
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        # hidden state
        hs[t] = np.tanh(W_xh@xs[t] + W_hh@hs[t-1] + b_h)
        # output (unnormalised potentials)
        ys[t] = W_hy@hs[t] + b_y
        # normalised probablities
        ps[t] = np.exp(ys[t])/np.sum(np.exp(ys[t]))
        # softmax (cross-entropy loss)
        loss += -np.log(ps[t][targets[t]][0])

    # backward pass
    dLdW_xh, dLdW_hh, dLdW_hy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
    dLdb_h, dLdb_y = np.zeros_like(b_h), np.zeros_like(b_y)
    dLdh_next = np.zeros_like(hs[0])
    for t in np.flip(np.arange(len(inputs)),0):
        # dL/dy
        dLdy = np.copy(ps[t])
        dLdy[targets[t]] -= 1
        # dL/dW_hy
        dLdW_hy += (dLdy@hs[t].T) # (v x 1) x (h x 1).T = (v x h)
        # dL/db_y
        dLdb_y += dLdy
        # dL/dh
        dLdh = W_hy.T@dLdy # (v x h).T x (v x 1) = (h x 1)
        # backprop through tanh
        dLdh_raw = (1 - hs[t]*hs[t]) * dLdh
        # dL/db_h
        dLdb_h += dLdh_raw
        # dL/dW_hh
        dLdW_hh = dLdh_raw@hs[t-1].T
        # dL/dW_xh
        dLdW_xh = dLdh_raw@xs[t].T
        # hidden gradient flow to the previous step h[t-1]
        dLdh_next = W_hh.T@dLdh_raw
    
    # clip to mitigate gradient vanish/explode
    for dparam in [dLdW_hy, dLdW_hh, dLdW_xh, dLdb_y, dLdb_h]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dLdW_xh, dLdW_hh, dLdW_hy, dLdb_h, dLdb_y, hs[len(inputs)-1]

In [10]:
def sample(h, seed_index, n):
    x = np.zeros((vocab_size, 1))
    x[seed_index] = 1
    indexes = []
    for t in np.arange(n):
        h = np.tanh(W_hh@h + W_xh@x + b_h)
        y = W_hy@h + b_y
        p = np.exp(y)/np.sum(np.exp(y))
        index = np.random.choice(np.arange(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[index] = 1
        indexes.append(index)
    return indexes

In [11]:
iterations = 100000
pointer = 0 # data pointer
mW_xh, mW_hh, mW_hy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mb_h, mb_y = np.zeros_like(b_h), np.zeros_like(b_y) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size) * seq_length # loss at iteration 0

for i in range(iterations):
    if pointer+seq_length+1 > len(data) or i == 0:
        hprev = np.zeros((hidden_size, 1)) # reset rnn memory
        pointer = 0
    inputs = [char_to_ix[ch] for ch in data[pointer:pointer+seq_length]]
    targets = [char_to_ix[ch] for ch in data[pointer+1:pointer+seq_length+1]]
    
    if i % 100 == 0:
        sample_indexes = sample(hprev, inputs[0], 100)
        txt = ''.join(ix_to_char[ix] for ix in sample_indexes)
        print('----\n {} \n----'.format(txt))

    loss, dLdW_xh, dLdW_hh, dLdW_hy, dLdb_h, dLdb_y, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    
    if i % 100 == 0:
        print('iter: {}, loss: {}'.format(i, smooth_loss))
        
    for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                [dLdW_xh, dLdW_hh, dLdW_hy, dLdb_h, dLdb_y], 
                                [mW_xh, mW_hh, mW_hy, mb_h, mb_y]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
    
    pointer += seq_length

----
  cJPaqVH] P Sy%R89lktLBz93fu]]"MpVrq],P7E[guBkV"JftvniKPD3]1(9wo)r
jdaCxuc8G:-TCpNznc9%KhvAzKxMH8a53 
----
iter: 0, loss: 105.48768672671387
----
 n-B6i tn3[ezpfiowi :qhvozamHtnco r u[fahesff[r8ie:yCeex7rVeeC8dhHi 8Lrle oyf-p G,reLV9chesc
o6eN irs 
----
iter: 100, loss: 106.23796968091015
----
 uaiasgeo Rt]r aawdLcTav2  i  e cca smarade HpaaNoeoit5 rgas omftguscqVmpNoeash1 kan d
iatuw nituiccC 
----
iter: 200, loss: 104.5417168464634
----
 eLiAr9hacinesa7nruvicfB,kN1 ho nzfhexea aP tN nnwnso
r-gwu Ldna ak narq d  bifasgonqmM y a mo[ h[c p 
----
iter: 300, loss: 102.98004394885234
----
 enGaies
HGaeo,ar. pre ol thbien aro(iRakztdcc p se'nioo4idp Gawioen sie leAnnpt aollevirisosNillpr i 
----
iter: 400, loss: 101.06159508269275
----
  cm1ref cittrafipVlo imi9netrdaltt 'r pi swerr ae epha oekneis  (gadfpSi n ieioraaitveiade]nd  Leuc9 
----
iter: 500, loss: 99.68632615054334
----
 i otdclicoditaataa u cMeenko nhvvcsgw aae6tea szJt tksme lswvenrcatun  sseic
oini ja pnctd eCcetosvp 
--

----
  bigecant to siae neris meae.dmasnetifaw as rarryerv-le oniret efsinvr
a tiam re  Air t kSorecanp is 
----
iter: 5600, loss: 77.06256137851197
----
 trs  auA 1ectAt aewa hmog Ppndor eeovdoen 7ats i]irqchoRiehe ti mudhue.gu [bsuochtasetyome gticibcri 
----
iter: 5700, loss: 77.15620854601322
----
 triiasdw dldthofurtat otmec.res I1ttiris ba2dofSsw mfrN ehnral  i as lp rae 1T aalnepn ir en [essstm 
----
iter: 5800, loss: 76.65682930305506
----
 eutrifane
ty bnidsec teteles 5e cNlv ]t wtd
 Rhigf rwer ef2oh bo.s ted vnin ke rolariiral ufinod.eo  
----
iter: 5900, loss: 76.80255813218129
----
 e ]e TotiuorkuthMial onvcw rbu pnos tasen thosrce Lorr Aongc iennotcismen ioeas dt a s)on ieriv arcg 
----
iter: 6000, loss: 76.32450369129215
----
 rtgod ko asdtmt serprbirectdaeu . phi  nh Ht]R"1 nhen aeh ea nerba,me 9 f
,oterguteedtelsat] onC ]8y 
----
iter: 6100, loss: 76.45152443489141
----
 si etl,oduliits Codtsiccgp si iice th0 on M aisfwonrnlg mtp coge Iwsnta.r,ssminroh profg xtinrincicp

----
 nelImtigt tinn Ecy Ipin tharg whrrerme:m
seciadatn th Brit a.ciobc)rg
LhMut rinlg oawitess Rorigetio 
----
iter: 11100, loss: 68.59495631524042
----
 ursens rels sot ras) tonin.9 Caedoedet a ch[vaslu ovache sicoren Khbhichergy th balerd emineds thtec 
----
iter: 11200, loss: 68.4435564449229
----
 eprra ssdr esthaimetayutioncicianonsw anp vhasp ratinsi s ro aay bocwhMit a.T thM) hctrarf as Ai emu 
----
iter: 11300, loss: 68.37888720448996
----
  ccomcin,t1t sodtho[e te nerwsserb iy okhersoputw n0onarrnsedshese tirrin it)oy lirenpfci cobyM[s th 
----
iter: 11400, loss: 68.24332102208422
----
 onsst aSs so:setwonk  ITy M re 44]erectiog Rga ren uan R07of amertherestas nethatinet. ueucemos e to 
----
iter: 11500, loss: 68.25815206611054
----
 qcaimedace.ore uomaligs by vov eciondisti wern't usp f,idt.tressi n an inporevye RiTe:cangemme  chss 
----
iter: 11600, loss: 68.13291653680437
----
 
1tesercalt ats
te'dsusirticiracind ovecbiindem toenenvtivenpchincionk i 58 lou aine tsort[,N'd

----
 

A943Fl aleccofncc)tites acornn tats.'ny ris Mgl ored Mivattit eo aag (bN aaly haspuaa fone pniteac 
----
iter: 16600, loss: 64.88212545515182
----
 onyporB itigecan atdockhetons.icTe n.y beutn cerwaydcati talsicenemaor ro ao fpr maycuin  rCe 20fel  
----
iter: 16700, loss: 64.75392127756517
----
 irthomsve th echoroobg tomeqwa (Ipnt
 ety 999Te weoche bF. ronecontiog rom g[6TM ton xN  sNitu tinti 
----
iter: 16800, loss: 64.84149771185443
----
 f wets eray on Lomedtetsagr bone  and ereihel[1]pstaoursigutpruawaksing nancorksingal-uesause aiqded 
----
iter: 16900, loss: 64.71177042622567
----
 nedorf  whay atsdore op arig.[1stispat  atsrew ivadoif nonga,ceatnel neeppftsitd had nh.
Bomn),todch 
----
iter: 17000, loss: 64.73604245429489
----
 fi he L
mC crthi hircu schprf ertiras a n thesivers yt Jersh n o0ch Ke ive dedr- acorl . Iw o nitspr 
----
iter: 17100, loss: 64.57538251817722
----
 deyn tey eraniogs [8Ts[1 TM oneconent simed  B aig   hgn eranda mnis eqthititnittha fedwiigneg

----
 mfit timN n  onakearn asta serla l tordon

whe--l  te biu b%rabf ta thel bors e 8one socse oime.[0F[ 
----
iter: 22100, loss: 63.193681153976115
----
 rap ie  anM, f 
T9 inedt ito n
 BTecsingtocn iinbN 0035,s Ln toredunsuan unn todeinedw sodecty,cipun 
----
iter: 22200, loss: 63.2829319231942
----
 edwprecarinelde sfetd98 meuseacaal ty trasciigug,t Hor-fN sitrong  iy gNinucartumletrsin san  puce h 
----
iter: 22300, loss: 63.14058052564315
----
 uind eaag endtiginc hoctei.loib .i-n u eusau Iagerm99 Fe sine 2
eq,itasd nstJa ivvva-vcelcwiss toriv 
----
iter: 22400, loss: 63.2662814287199
----
 nor fofnate tereet hg oor escakrtorriodet ay it[wmer iapunrindvanne tite w oal orn isenl as aninecti 
----
iter: 22500, loss: 63.13283047839972
----
 [3] %e wobk unathncbact tha ineval ors uts ttadiveorol fecbyivndroncen Len tthee (Npung uetwermoml   
----
iter: 22600, loss: 63.269544041878504
----
 i uaines  yi tica tencork cthotl Geqcha tcho fasnrouschetnutpcietl-ttby
aot ec eutarnereods.ie

...

----
 an tr aayTute tinivwhapks[23[23s,it aryc ao4d 
r Tsivtal 20"etiny apyiow whM srfanry aliedonfle (una 
----
iter: 77100, loss: 60.49015125963916
----
 n t ce tim Rtivre rega tpsivatesrnerigrreress ones tyargeds ainel, aneucoll sireworoll siid re 
Line 
----
iter: 77200, loss: 60.382672641443506
----
 pn,tttue eroesprmiti pusiti gonsict nmariss sengatanen thtteis tr sichgtes rto  fma-n nrhua verwaNs  
----
iter: 77300, loss: 60.41497774918708
----
  coci seqwhegil.l sy "in segworkaipltonalt gal offeads1cionn tonkeul Ti s,s afsia] a Nivtos dom nr   
----
iter: 77400, loss: 60.36288383374514
----
 tiivs,utcorther8ornctvonetirn nat twotss at  Ls treninnt bxttsapntsing
L000du-  Inesedsisisg th Mron 
----
iter: 77500, loss: 60.44185814591649
----
 evine noviormals efuetthwonggid bla oo cesseut (LSasrarn diue womeal somshent an be] aa  Saippls Apu 
----
iter: 77600, loss: 60.356395014046164
----
 .onst,rtsivat aokelygine ce poines alithat sgie-crongusu nuan in Re fom (epirce whel Citd ts

----
 nopeturaead4ss Eeit th o socit pracor f teq atucbitwit any

nos al Tnasey ealien (Busdnsen LN Tomaic 
----
iter: 82600, loss: 60.1961620371534
----
 edoswiassitsempubgc Ae e cuodss fNN reine jads u att.trbpuro s aee G sonp
it eaProl efco Leavelp so  
----
iter: 82700, loss: 60.32402804166828
----
 nge timet ang wto esel Teseroa rere sims  toTel oatsily tna5, orel ore pns)it ores Horen copuof eofu 
----
iter: 82800, loss: 60.20739238943775
----
 ut  CNNingthe i ch onl itpingork roetl tanil[8yNivg  auls fing actirferneas
tiystofkel uas sofbpigin 
----
iter: 82900, loss: 60.35552283786655
----
 n)' Bas te uharken 
ial dat ofs the corucoanl [20]uT aniig  sra ion 
MN,re thanvumet Llite  Ronul ce 
----
iter: 83000, loss: 60.23949345649641
----
 eu in 1irCcatiit aiget bofgethedsska ]anvormetir g us arsorg tes ofe
uTdosketpsser n b593h-ma e s ue 
----
iter: 83100, loss: 60.397704006007025
----
 s rom Tyecly) an sagan)cGefuneed]telp in trro rornerps nst chocovnet To th ed erm thes orem if

----
 ne tTiTond prorret arivtwotis ss ofechatinec ]i
tey 1Nuetoerpic, al rin ertovolNddCgam rtis. I ined  
----
iter: 88100, loss: 60.397975548082876
----
 niciovne  Redn y
CAlacbonrt Mes)s ent Hby (A ohrectsacin s etit),s Fee iongetrchet tortuats rorss ro 
----
iter: 88200, loss: 60.335009585802176
----
  stione-vrodrs.

s neso feicegaovm in icta morkivgeavoui lN nal teci.[1hsiot exnct St. m iu intiat a 
----
iter: 88300, loss: 60.40671633648846
----
 e wome Se cemree Pts uiivernorkotwerseuctohn  LnMNy l ben by 2sege sive tie uaet anst wovhnputit[1]T 
----
iter: 88400, loss: 60.32357208401581
----
 iis s iuha  nN (Canirg ars tenMgedofdoed fuel ra]pflitowg Crcthy L h eemitecat[1rkit anptmitcoud rec 
----
iter: 88500, loss: 60.37176565568308
----
  is ofinal[1akiue
norceisi.
Frftrech ttiseqca tuecas nsprfbM-conice de. feocen thmage R etherid ermm 
----
iter: 88600, loss: 60.269682632459535
----
 icMing bwhin.Tt Cua 1uce ogwosieuctems ay  Seae sa b monfss.

I w entie ex sovopeaN.[1 Iwov

----
 ln ns,orthaorsivigue an ues Ma nedco .
Ituh tsocn ertie dorkbed ne uon tuceersotipermae an t troe.di 
----
iter: 93600, loss: 60.331794508032395
----
 ttho n enpnaingtr23hettrhedetal eec ay 
RSa aani vhri shssac(TSit.. Heive je tcmitianei.ichtite recb 
----
iter: 93700, loss: 60.3123280864985
----
  Sal ene 8an n-n igeenece cwarageucwononuet]och[ Leduats et ierignicainnte mecpumocurert unitideches 
----
iter: 93800, loss: 60.32403819774406
----
 a pu][78]ed iet Be tomn mensireint the bhi 1hed erog l by pr ychntha-l fopas-cala setdhTy Lns[1]ans  
----
iter: 93900, loss: 60.28063336054292
----
 f wreluch  at-mM dhbpns as a se nonm oonnitpsrtioningorf) ([d]s
thmon cayg sphh rnem te-cating. Rrau 
----
iter: 94000, loss: 60.303632813617874
----
 ss wnhe ferererdol BNN ac orrsope t
in recitsniongget,doprmsined trmrdfdsa[1osapdtig.[1]]anlctitrane 
----
iter: 94100, loss: 60.295032641968014
----
 yp tpas the jNhy,vges ofn maraure uedc ih by tof edsim nor oikta 
Crocf oe bl  aaccathswarop

----
  b  mtoar t tncopral,ve uea  spune perditi auch an ofinst asputettorce fofweirig.dernievso si trhMTo 
----
iter: 99100, loss: 59.66356308918064
----
 non ue suca eqt headl ofnut aitp Hs,artims oceiteoutt)whs on itnpercs,me Sonfainuis ronel ogfhe orye 
----
iter: 99200, loss: 59.69695674825262
----
 uras ertetpuTes,  Teivad by res afd meivtsuucorta tpciopvoec(wiofs encodce ertenges ettorb urfokal L 
----
iter: 99300, loss: 59.62538297525612
----
 tingeepu-gata.d jtattonv ornl inid aigfian uoch oVsirgedcsive Swec cTh)]ee- caty (wwcethed,sgnon-git 
----
iter: 99400, loss: 59.6863251183951
----
 asu  ouctals veechemay rhn [1herern suetarm ecortarnons soq iel Rsheneefig anl w emuTM't. Ierousip7t 
----
iter: 99500, loss: 59.6565208540003
----
 veineduta thoneinegseconon prov an rLoui acasdchov te Laitino[cngemas), marfinguen whed ech etreppls 
----
iter: 99600, loss: 59.7150291077908
----
 vepual eolie.[mapcreiod ssrory Cwome-uvio sb-ld Tnentrs (Np t Lautwect ceruby becanyi-nded rerrai